# Segmentation and Tracking

This notebook demonstrates a standard segmentation and tracking workflow.
It loads a vector of example images,
segments them,
then runs a tracking algorithm to match floes in the images.

In this example, we use two images.
However, the same syntax can be used for a larger number of images. 


Set up the environment.

In [ ]:
using Pkg
Pkg.add("IceFloeTracker")
Pkg.add("Images")

Load packages and functions required. 

In [ ]:
using IceFloeTracker: LopezAcosta2019Tiling, Watkins2026Dataset, FloeTracker, FilterFunction, MinimumWeightMatchingFunction, modis_truecolor, modis_falsecolor, modis_landmask, info
using Images: erode, segment_mean, labels_map, SegmentedImage, RGB, mosaicview
using Dates: DateTime

Load the images to be segmented. 
In this case we use data from the Watkins 2026 Ice Floe Validation dataset.

In [ ]:
dataset = Watkins2026Dataset(; ref="v0.1")
filter!(c -> c.case_number == 6, dataset)

For some filtering algorithms, 
the pass-times of the observations need to be in ascending order,
so we sort by the pass-time.

In [ ]:
sort!([:pass_time], dataset)

View the data:

In [ ]:
mosaicview([modis_truecolor.(dataset); modis_falsecolor.(dataset); modis_landmask.(dataset)], nrow=2)

The metadata for the dataset are:

In [ ]:
info(dataset)

Run segmentation on the images.

In [ ]:
segmenter = LopezAcosta2019Tiling.Segment()
segmentation_results = segmenter.(modis_truecolor.(dataset), modis_falsecolor.(dataset), modis_landmask.(dataset))

Run the tracker on the segmentation results.
The tracker takes a vector of segmented images 
and requires the observation time of each for filtering candidate matches.

In [ ]:
tracker = FloeTracker(;
    filter_function=FilterFunction(),
    matching_function=MinimumWeightMatchingFunction()
)
tracking_results = tracker([segmentation_results[1], segmentation_results[1]], info(dataset).pass_time)

The tracking results are returned as a dataframe of floes and their matches. 

In [ ]:
tracker = FloeTracker(;
    filter_function=FilterFunction(),
    matching_function=MinimumWeightMatchingFunction()
)
tracking_results = tracker(reverse(segmentation_results), info(dataset).pass_time)